# 🎬 Semantic A-Roll/B-Roll Engine - Google Colab Backend

**Problem:** Local Windows system crashes with PyTorch  
**Solution:** Run backend on Colab's free GPU!

## Setup (Run these cells in order)

In [ ]:
# Cell 1: Install dependencies
!pip install -q fastapi uvicorn pydantic python-multipart
!pip install -q google-cloud-aiplatform
!pip install -q faiss-cpu numpy pillow
!pip install -q torch torchaudio
!pip install -q git+https://github.com/m-bain/whisperx.git
!apt-get install -qq ffmpeg

print("✅ Dependencies installed!")

In [ ]:
# Cell 2: Clone your code (or upload files)
import os
from google.colab import files

# Option A: Upload your backend folder as ZIP
print("Upload your 'backend' folder as a .zip file:")
uploaded = files.upload()

# Extract
!unzip -q backend.zip
print("✅ Backend code uploaded!")

In [ ]:
# Cell 3: Setup GCP credentials
from google.colab import auth
auth.authenticate_user()

# Set environment variables
os.environ['GCP_PROJECT_ID'] = 'firstproject-c5ac2'
os.environ['GCP_LOCATION'] = 'us-central1'
os.environ['WHISPER_DEVICE'] = 'cuda'  # Use Colab's GPU!
os.environ['CUDA_DEVICE'] = 'cuda'

print("✅ GCP authenticated!")

In [ ]:
# Cell 4: Create directories
!mkdir -p uploads outputs data models/whisperx
print("✅ Directories created!")

In [ ]:
# Cell 5: Install ngrok to expose API
!pip install -q pyngrok
from pyngrok import ngrok

# Get your ngrok authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("YOUR_NGROK_TOKEN_HERE")  # ← Replace this!

print("✅ Ngrok configured!")

In [ ]:
# Cell 6: Start FastAPI backend
import nest_asyncio
import uvicorn
from threading import Thread

nest_asyncio.apply()

# Start uvicorn in background thread
def run_server():
    uvicorn.run(
        "backend.main:app",
        host="0.0.0.0",
        port=8000,
        log_level="info"
    )

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

import time
time.sleep(5)  # Wait for server to start

print("✅ FastAPI server started!")

In [ ]:
# Cell 7: Expose with ngrok
public_url = ngrok.connect(8000)

print(f"\n🌐 Your public API URL:")
print(f"   {public_url}")
print(f"\n📝 Copy this URL and use it in your local Streamlit frontend!")
print(f"\n   Change API_BASE in frontend/app.py to:")
print(f"   API_BASE = '{public_url}'")

In [ ]:
# Cell 8: Test with JSON
import requests
import json

# Your video JSON
video_json = {
    "a_roll": {
        "url": "https://fzuudapb1wvjxbrr.public.blob.vercel-storage.com/food_quality_ugc/a_roll.mp4",
        "metadata": "Food quality awareness"
    },
    "b_rolls": [
        {
            "id": "broll_1",
            "url": "https://fzuudapb1wvjxbrr.public.blob.vercel-storage.com/food_quality_ugc/broll_1.mp4",
            "metadata": "Mumbai street food"
        }
        # Add all 6 B-Rolls here...
    ]
}

# Send to backend
response = requests.post(
    f"{public_url}/api/process/json",
    json=video_json
)

job_id = response.json()['job_id']
print(f"✅ Processing started! Job ID: {job_id}")

# Monitor status
while True:
    status = requests.get(f"{public_url}/api/status/{job_id}").json()
    print(f"Status: {status['status']} - {status['progress']}% - {status['message']}")
    
    if status['status'] in ['complete', 'error']:
        break
    time.sleep(5)

if status['status'] == 'complete':
    print("\n🎉 Processing complete!")
    print(f"   Download: {public_url}/api/download/{job_id}")
else:
    print(f"\n❌ Error: {status.get('error', 'Unknown')}")

## Keep Session Alive

Run this cell to prevent Colab from disconnecting:

In [ ]:
# Keep alive script
import time
while True:
    print("✅ Backend running...", end='\r')
    time.sleep(60)